# <center>NLP: Topic Modeling for Riyadh Newspaper Articles</center>



**DATA**: We are planning to use our previously **scraped** data from the Riyadh Newspaper website **in Arabic**, ~3GB, plus some additional data (MetaData) that could add a meaningful sense to our project, see [Riyadh Newspaper website](http://www.alriyadh.com/1814297) to understand how we scraped the data.

**EXPECTED OUTPUT**: At the end of this project we expect to be done NLP **Topic Modeling** on Riyadh Newspaper articles, and all it's required preproccesing steps in addition to approperiate visualization.

**BONUS TASKS**: 
- Scraping ~3GB Data. 
- Using SQL Database for data storage. 
- As a challenge, we used data written in Arabic.

# 1. Data

In [33]:
import pandas  as pd
import numpy as np
import pyodbc

import nltk # pip install nltk
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.stem.isri import ISRIStemmer

from sklearn.feature_extraction.text import TfidfVectorizer


### reading from Database

In [2]:
# server = 't5.database.windows.net'
# database = 'T5'
# username = 't5'
# password = 'My404Data'

# # dfs2.columns=dfs2.columns.str.strip()

# cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
# cursor = cnxn.cursor()

In [3]:
# df= pd.read_sql("select * from articles",cnxn)
# df
# df.to_csv("articles.csv", encoding='utf-8-sig')

In [4]:
# df2= pd.read_sql("select top 10000 * from texts",cnxn)
# df2
# df2.to_csv("texts.csv",encoding='utf-8-sig')

### reading CSVs files

In [5]:
df_articles = pd.read_csv('../articles.csv')

In [6]:
df_texts = pd.read_csv('../texts.csv')

### preveiw

In [7]:
df_articles.head(5)

,Unnamed: 0,A_ID,A_Link,A_LinkNum,A_Timestr,A_Cat,A_Auth,A_Title1,A_Title2,A_Paragraphs,A_Words,A_Characters
0,0,1,http://www.alriyadh.com/1814297,1814297,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,"الرياض - ""الرياض""",إلى ما بعد اجتماع مجموعة أوبك بلس,السعودية تؤجل الإعلان عن أسعار الخام لشهر مايو,8,339,1962
1,1,2,http://www.alriyadh.com/1814297,1814297,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,"الرياض - ""الرياض""",إلى ما بعد اجتماع مجموعة أوبك بلس,السعودية تؤجل الإعلان عن أسعار الخام لشهر مايو,8,339,1962
2,2,3,http://www.alriyadh.com/1814296,1814296,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,وادي الدواسر - سعود آل مسيب,NaN,تعليم وادي الدواسر يبدأ المرحلة الثانية من خطة...,3,183,1162
3,3,4,http://www.alriyadh.com/1814296,1814296,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,وادي الدواسر - سعود آل مسيب,NaN,تعليم وادي الدواسر يبدأ المرحلة الثانية من خطة...,3,183,1162
4,4,5,http://www.alriyadh.com/1814295,1814295,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,الرياض - محمد الحيدر,من برنامج إعانة الباحثين عن عمل,"""هدف"" يودع 446 مليون ريال في حسابات المستفيدين",4,190,1148


In [8]:
df_texts.head(5)

,Unnamed: 0,T_ID,T_AID,T_Text
0,0,87160,174309,\nعلى الرغم من التحذيرات الشديدة والمتكررة الت...
1,1,87164,174317,\nأدى محافظ ضمد عبدالله خالد البراق عقب صلاة ع...
2,2,87165,174319,\nرفع رئيس مجلس إدارة الهيئة العامة للولاية عل...
3,3,87168,174325,\nوجه صاحب السمو الملكي الأمير د. فيصل بن مشعل...
4,4,87169,174327,\nاستقبل صاحب السمو الملكي الأمير فيصل بن خالد...


# Article Example

In [9]:
df_texts['T_Text'][0]

'\nعلى الرغم من التحذيرات الشديدة والمتكررة التي يوجهها الدفاع المدني للمواطنين قبل وبعد هطول الأمطار لتوخي الحذر والابتعاد عن أماكن تجمعات السيول وبطون الأودية ورغم ما تتناقله وسائل التواصل الاجتماعي من مقاطع مؤلمة ومفجعة لأشخاص جرفتهم السيول بمركباتهم ولقوا حتفهم بأبشع الصور وأسوأء المناظر، إلاّ أنه مازال هناك الكثير من المراهقين ومن فاقدي البصيرة المتهورين تستهويهم المغامرات المهلكة داخل بطون الأودية والأماكن التي غمرتها مياه الأمطار، غير مبالين بخطورة تلك الأفعال وشناعة تلك التصرفات العبثية، التي ربما يفقدون حياتهم بسببها في أدنى لحظة.\nوفي كل موسم من مواسم الأمطار تتجدد ظاهرة التهور والمغامرات في السيول وتتجدد معها معاناة رجال الدفاع المدني الذين مهما كان لديهم من إمكانات لن يستطيعوا تغطية جزء مساحته مئات الكيلومترات من براري المملكة الشاسعة التي تتنوع فيها التضاريس المعقدة.\nماذا بعد\nوتداولت شبكات التواصل الاجتماعي مقاطع فيديوهات لبعض المتهورين وهم يقتحمون الأودية مع سبق الإصرار، غير مبالين بتحذيرات الدفاع المدني وغير مبالين بالأشخاص الموجودين بتلك المواقع الذين حاولوا ثنيهم عن 

# 2. Preprocessing

In [21]:
text = df_texts['T_Text'][0]
text

'\nعلى الرغم من التحذيرات الشديدة والمتكررة التي يوجهها الدفاع المدني للمواطنين قبل وبعد هطول الأمطار لتوخي الحذر والابتعاد عن أماكن تجمعات السيول وبطون الأودية ورغم ما تتناقله وسائل التواصل الاجتماعي من مقاطع مؤلمة ومفجعة لأشخاص جرفتهم السيول بمركباتهم ولقوا حتفهم بأبشع الصور وأسوأء المناظر، إلاّ أنه مازال هناك الكثير من المراهقين ومن فاقدي البصيرة المتهورين تستهويهم المغامرات المهلكة داخل بطون الأودية والأماكن التي غمرتها مياه الأمطار، غير مبالين بخطورة تلك الأفعال وشناعة تلك التصرفات العبثية، التي ربما يفقدون حياتهم بسببها في أدنى لحظة.\nوفي كل موسم من مواسم الأمطار تتجدد ظاهرة التهور والمغامرات في السيول وتتجدد معها معاناة رجال الدفاع المدني الذين مهما كان لديهم من إمكانات لن يستطيعوا تغطية جزء مساحته مئات الكيلومترات من براري المملكة الشاسعة التي تتنوع فيها التضاريس المعقدة.\nماذا بعد\nوتداولت شبكات التواصل الاجتماعي مقاطع فيديوهات لبعض المتهورين وهم يقتحمون الأودية مع سبق الإصرار، غير مبالين بتحذيرات الدفاع المدني وغير مبالين بالأشخاص الموجودين بتلك المواقع الذين حاولوا ثنيهم عن 

### 2.2. Lower casing

In [22]:
text = text.lower()
text

'\nعلى الرغم من التحذيرات الشديدة والمتكررة التي يوجهها الدفاع المدني للمواطنين قبل وبعد هطول الأمطار لتوخي الحذر والابتعاد عن أماكن تجمعات السيول وبطون الأودية ورغم ما تتناقله وسائل التواصل الاجتماعي من مقاطع مؤلمة ومفجعة لأشخاص جرفتهم السيول بمركباتهم ولقوا حتفهم بأبشع الصور وأسوأء المناظر، إلاّ أنه مازال هناك الكثير من المراهقين ومن فاقدي البصيرة المتهورين تستهويهم المغامرات المهلكة داخل بطون الأودية والأماكن التي غمرتها مياه الأمطار، غير مبالين بخطورة تلك الأفعال وشناعة تلك التصرفات العبثية، التي ربما يفقدون حياتهم بسببها في أدنى لحظة.\nوفي كل موسم من مواسم الأمطار تتجدد ظاهرة التهور والمغامرات في السيول وتتجدد معها معاناة رجال الدفاع المدني الذين مهما كان لديهم من إمكانات لن يستطيعوا تغطية جزء مساحته مئات الكيلومترات من براري المملكة الشاسعة التي تتنوع فيها التضاريس المعقدة.\nماذا بعد\nوتداولت شبكات التواصل الاجتماعي مقاطع فيديوهات لبعض المتهورين وهم يقتحمون الأودية مع سبق الإصرار، غير مبالين بتحذيرات الدفاع المدني وغير مبالين بالأشخاص الموجودين بتلك المواقع الذين حاولوا ثنيهم عن 

### 2.1. Tokenization

In [23]:
text = ' '.join(word_tokenize(text))
# sent_tokenize(df_texts['T_Text'][0])
text

'على الرغم من التحذيرات الشديدة والمتكررة التي يوجهها الدفاع المدني للمواطنين قبل وبعد هطول الأمطار لتوخي الحذر والابتعاد عن أماكن تجمعات السيول وبطون الأودية ورغم ما تتناقله وسائل التواصل الاجتماعي من مقاطع مؤلمة ومفجعة لأشخاص جرفتهم السيول بمركباتهم ولقوا حتفهم بأبشع الصور وأسوأء المناظر، إلاّ أنه مازال هناك الكثير من المراهقين ومن فاقدي البصيرة المتهورين تستهويهم المغامرات المهلكة داخل بطون الأودية والأماكن التي غمرتها مياه الأمطار، غير مبالين بخطورة تلك الأفعال وشناعة تلك التصرفات العبثية، التي ربما يفقدون حياتهم بسببها في أدنى لحظة . وفي كل موسم من مواسم الأمطار تتجدد ظاهرة التهور والمغامرات في السيول وتتجدد معها معاناة رجال الدفاع المدني الذين مهما كان لديهم من إمكانات لن يستطيعوا تغطية جزء مساحته مئات الكيلومترات من براري المملكة الشاسعة التي تتنوع فيها التضاريس المعقدة . ماذا بعد وتداولت شبكات التواصل الاجتماعي مقاطع فيديوهات لبعض المتهورين وهم يقتحمون الأودية مع سبق الإصرار، غير مبالين بتحذيرات الدفاع المدني وغير مبالين بالأشخاص الموجودين بتلك المواقع الذين حاولوا ثنيهم عن رأي

### 2.3. Stop words removal

In [28]:
# Hint: we can try spaCy
sw_nltk = stopwords.words('arabic')

text = [word for word in text.split() if word.lower() not in sw_nltk]
# text = " ".join(text)
text

['الرغم',
 'التحذيرات',
 'الشديدة',
 'والمتكررة',
 'يوجهها',
 'الدفاع',
 'المدني',
 'للمواطنين',
 'وبعد',
 'هطول',
 'الأمطار',
 'لتوخي',
 'الحذر',
 'والابتعاد',
 'أماكن',
 'تجمعات',
 'السيول',
 'وبطون',
 'الأودية',
 'ورغم',
 'تتناقله',
 'وسائل',
 'التواصل',
 'الاجتماعي',
 'مقاطع',
 'مؤلمة',
 'ومفجعة',
 'لأشخاص',
 'جرفتهم',
 'السيول',
 'بمركباتهم',
 'ولقوا',
 'حتفهم',
 'بأبشع',
 'الصور',
 'وأسوأء',
 'المناظر،',
 'إلاّ',
 'أنه',
 'الكثير',
 'المراهقين',
 'فاقدي',
 'البصيرة',
 'المتهورين',
 'تستهويهم',
 'المغامرات',
 'المهلكة',
 'داخل',
 'بطون',
 'الأودية',
 'والأماكن',
 'غمرتها',
 'مياه',
 'الأمطار،',
 'مبالين',
 'بخطورة',
 'الأفعال',
 'وشناعة',
 'التصرفات',
 'العبثية،',
 'ربما',
 'يفقدون',
 'حياتهم',
 'بسببها',
 'أدنى',
 'لحظة',
 '.',
 'وفي',
 'موسم',
 'مواسم',
 'الأمطار',
 'تتجدد',
 'ظاهرة',
 'التهور',
 'والمغامرات',
 'السيول',
 'وتتجدد',
 'معها',
 'معاناة',
 'رجال',
 'الدفاع',
 'المدني',
 'لديهم',
 'إمكانات',
 'يستطيعوا',
 'تغطية',
 'جزء',
 'مساحته',
 'مئات',
 'الكيلومترات',
 'براري',

### 2.4. Stemming or Lemmatization

In [36]:
st = ISRIStemmer()

for w in text:
    print(w, " : ", st.stem(w))
    new_text += st.stem(w)
text = new_text

الرغم  :  رغم
التحذيرات  :  حذر
الشديدة  :  شدد
والمتكررة  :  تكرر
يوجهها  :  يوج
الدفاع  :  دفع
المدني  :  دني
للمواطنين  :  وطن
وبعد  :  بعد
هطول  :  هطل
الأمطار  :  مطر
لتوخي  :  لتخ
الحذر  :  حذر
والابتعاد  :  بعد
أماكن  :  اما
تجمعات  :  جمع
السيول  :  سيل
وبطون  :  وبط
الأودية  :  ودي
ورغم  :  رغم
تتناقله  :  نقل
وسائل  :  وسل
التواصل  :  وصل
الاجتماعي  :  جمع
مقاطع  :  قطع
مؤلمة  :  ؤلم
ومفجعة  :  فجع
لأشخاص  :  أشخاص
جرفتهم  :  جرف
السيول  :  سيل
بمركباتهم  :  ركب
ولقوا  :  ولق
حتفهم  :  حتف
بأبشع  :  أبشع
الصور  :  صور
وأسوأء  :  أسوأء
المناظر،  :  مناظر،
إلاّ  :  الا
أنه  :  انه
الكثير  :  كثر
المراهقين  :  رهق
فاقدي  :  فقد
البصيرة  :  بصر
المتهورين  :  تهر
تستهويهم  :  سهو
المغامرات  :  غمر
المهلكة  :  هلك
داخل  :  دخل
بطون  :  بطن
الأودية  :  ودي
والأماكن  :  اما
غمرتها  :  غمر
مياه  :  ياه
الأمطار،  :  طر،
مبالين  :  بال
بخطورة  :  خطر
الأفعال  :  فعل
وشناعة  :  شنع
التصرفات  :  صرف
العبثية،  :  عبثية،
ربما  :  ربم
يفقدون  :  فقد
حياتهم  :  حيت
بسببها  :  سبب
أدنى  :  دنى

# 3. Encoding (???, TFIDF or Bag of Word)

### 3.1. ???

### 3.2. TFIDF

In [38]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([text, text])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df

,أثناء,أحيانا,أخبارا,أخي,أدنى,أرواحهم,أساسيا,أضف,أماكن,أمانة,...,يفعله,يفقدون,يقتحمون,يقوم,يكن,يكنمحبشغلناسهمك,يما,ينم,يهرع,يوجهها
0,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,...,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533
1,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,...,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533,0.035533


### 3.3. Bag of Word

# 4. Topic Modeling

### 4.1. LDA

### 4.2. ???

# 5. Clustering